## map/reduce
map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回。

举例说明

In [1]:
def f(x):
    return x*x
r = map(f,[1,2,3,4,5,6,7,8,9])
list(r)

[1, 4, 9, 16, 25, 36, 49, 64, 81]

map()传入的第一个参数是f，即函数对象本身。由于结果r是一个Iterator，Iterator是惰性序列，因此通过list()函数让它把整个序列都计算出来并返回一个list。
map()作为高阶函数，事实上它把运算规则抽象了，因此，我们不但可以计算简单的$f(x)=x^{2}$，还可以计算任意复杂的函数，比如，把这个list所有数字转为字符串：

In [2]:
 list(map(str, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

['1', '2', '3', '4', '5', '6', '7', '8', '9']

　　再看reduce的用法,reduce把一个函数作用在一个序列$[x_{1},x_{2},x_{3},...]$上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算，其效果就是：  
                  <center>reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4) </center> 
　　对一个序列求和，就可以用reduce实现：

In [3]:
from functools import reduce
def add(x,y):
    return x + y
reduce(add,[1,3,5,7,9])

25

　　如果要把序列[1, 3, 5, 7, 9]变换成整数13579，reduce就可以派上用场：

In [4]:
from functools import reduce
def fn(x,y):
    return 10 * x + y
reduce(fn,[1,3,5,7,9])

13579

考虑到字符串str也是一个序列，对上面的例子稍加改动，配合map()，我们就可以写出把str转换为int的函数：

In [5]:
from functools import reduce
def fn(x, y):
    return x * 10 + y
def char2num(c):
    d = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
    return d[c]
reduce(fn,map(char2num,'13579'))

13579

## filter
Python内建的filter()函数用于过滤序列。

和map()类似，filter()也接收一个函数和一个序列。和map()不同的是，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

例如，在一个list中，删掉偶数，只保留奇数，可以这么写：

In [6]:
def is_odd(n):
    return n%2 == 1
list(filter(is_odd,[1,2,3,4,5,6,7,8,9,12,13]))

[1, 3, 5, 7, 9, 13]

把一个序列中的空字符串删掉，可以这么写：

In [7]:
def not_empty(s):
    return s and s.strip()
list(filter(not_empty, ['A', '', 'B D ', None, 'C', '  ','zwx']))

['A', 'B D ', 'C', 'zwx']

注意到filter()函数返回的是一个Iterator，也就是一个惰性序列，所以要强迫filter()完成计算结果，需要用list()函数获得所有结果并返回list

### 用filter求素数：埃氏筛法

In [8]:
def _odd_iter():
    n = 1
    while True:
        n = n + 2
        yield n  #yield 的作用就是把一个函数变成一个 generator
        
def _not_divisible(n):
    return lambda x: x % n > 0

def primes():
    yield 2
    it = _odd_iter() # 初始序列
    while True:
        n = next(it) # 返回序列的第一个数
        yield n
        it = filter(_not_divisible(n), it) # 构造新序列
        
# 打印30以内的素数:
for n in primes():
    if n < 30:
        print(n)
    else:
        break

2
3
5
7
11
13
17
19
23
29


## 函数作为返回值
可变参数的求和，不需要立刻求和，而是在后面的代码中，根据需要再计算

In [9]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

f = lazy_sum(1, 3, 5, 7, 9)
f()

25

在这个例子中，我们在函数lazy_sum中又定义了函数sum，并且，内部函数sum可以引用外部函数lazy_sum的参数和局部变量，当lazy_sum返回函数sum时，相关参数和变量都保存在返回的函数中，这种称为“闭包（Closure）”的程序结构拥有极大的威力。

请再注意一点，当我们调用lazy_sum()时，每次调用都会返回一个新的函数，即使传入相同的参数：

In [10]:
f1 = lazy_sum(1, 3, 5, 7, 9)
f2 = lazy_sum(1, 3, 5, 7, 9)
f1==f2

False

### 闭包
注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。  
<font color = #ff0000>返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。</font>  
